In [ ]:
# Loading data

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
DF=pd.concat([train,test])

In [ ]:
# Decoding names
def decodeTitle(name):
    signif_titles=['Mr.','Miss.','Mrs.','Master.'] 
    # cut exactly title from column Name
    title=name[name.find(',')+2:name.find('.')+1]
    if title in signif_titles:  return(title)
    else: return('Arist.')
    
DF['Title']=DF['Name'].apply(decodeTitle)

In [ ]:
# Decodng cabins
def number(string):
    if pd.notnull(string):
        numbers=[int(i) for i in re.findall('(\d+)',string)]
        return(np.mean(numbers))
def symbol(string):
    if pd.notnull(string):
        return(True)
    else: return(False)

DF['CabinSymbol'] = DF.Cabin.apply(symbol)
DF['CabinNumber'] = DF.Cabin.apply(number)

In [ ]:
Ages, Fares=[0],[0]
for i in [1,2,3]:
    Ages.append(DF[DF.Pclass==i]['Age'].mean())
    Fares.append(DF[DF.Pclass==i]['Fare'].mean())
def funca(row):
    if row[0]>0: 
        age=row[0]
    else: 
        age=Ages[row[7]]
    return(age)
def funcb(row):
    if row[3]>0: 
        fare=row[3]
    else: 
        fare=Fares[row[7]]
    return(fare)
                 
DF['Age']=DF.apply(funca,axis=1)
DF['Fare']=DF.apply(funcb,axis=1)

In [ ]:
DF['Title']=DF['Title'].astype('category').cat.codes
DF['CabinSymbol']=DF['CabinSymbol'].astype('category').cat.codes
DF['Sex']=DF['Sex'].astype('category').cat.codes

In [ ]:
from sklearn.utils import shuffle
DF = shuffle(DF, random_state = 187)

In [ ]:
DF_train=DF[DF.Survived.notnull()]
DF_test=DF[DF.Survived.notnull()==False]
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

optimizer = GridSearchCV(RandomForestClassifier(), \
                         {'max_features' : [0.4, 0.6, 0.8, 1.0], 
                          'max_depth' : [5, 10, 15, 25, 35, 45, 55, 65], 
                          'n_estimators' : [1, 5, 10, 15, 30, 50, 70, 90, 100]},\
                         cv = 15, \
                         n_jobs=-1)
optimizer.fit(DF_train[['Age','Fare','SibSp','Parch','Pclass','Title','CabinSymbol','Sex']], DF_train['Survived'])
print('Random Forest Best Result', optimizer.best_score_)
print('Best Parameters', optimizer.best_params_)

In [ ]:
def writeAnswer(filename, optimizer):
    prediction = optimizer.predict(np.hstack([DF_test[['Age','Fare','SibSp','Parch','Pclass','Title','CabinSymbol','Sex']]]))
    submission = pd.DataFrame({
            "PassengerId": test["PassengerId"],
            "Survived": prediction
        })
    submission.Survived=submission.Survived.astype('int64',copy=False)
    submission.to_csv(filename, index=False)
    
writeAnswer('RandomForestClassifier.csv', optimizer)

In [ ]:
### Hand-made tree of solving
i1,i2,i3,i4,i5,i6,i7,i8,i9,i10,i11=0,0,0,0,0,0,0,0,0,0,0
j1,j2,j3,j4,j5,j6,j7,j8,j9,j10,j11=0,0,0,0,0,0,0,0,0,0,0
def funct(row):
    global i1,i2,i3,i4,i5,i6,i7,i8,i9,i10,i11
    global j1,j2,j3,j4,j5,j6,j7,j8,j9,j10,j11
    if row[8]=='female' and row[7] in [1,2]:
        j1+=1
        if row[10]==0: i1+=1
        return(1)
    elif row[8]=='female' and row[7]==3 and row[2]=='S':
        j2+=1
        if row[10]==1: i2+=1
        return(0)
    elif row[8]=='female' and row[7]==3 and row[2]!='S':
        j3+=1    
        if row[10]==0: i3+=1
        return(1)
    elif row[12]=='Master.' and row[7] in [1,2]: 
        j4+=1    
        if row[10]==0: i4+=1
        return(1)
    elif row[12]=='Master.' and row[7]==3 and row[3]<21:
        j5+=1
        if row[10]==0: i5+=1
        return(1)
    elif row[12]=='Master.' and row[7]==3 and row[3]>21:
        j6+=1     
        if row[10]==1: i6+=1
        return(0)
    elif row[8]=='male' and row[7]==3:
        j7+=1
        if row[10]==1: i7+=1
        return(0)
    elif row[8]=='male' and row[12]=='Arist.' and row[13]==True:
        j8+=1
        if row[10]==0: i8+=1
        return(1)
    elif row[8]=='male' and row[12]=='Arist.' and row[13]==False:
        j9+=1        
        if row[10]==1: i9+=1
        return(0)
    elif row[8]=='male' and row[7] in [1,2] and row[12]!='Master.'and row[3]<100:
        j10+=1
        if row[10]==1: i10+=1
        return(0)
    elif row[8]=='male' and row[7] in [1,2] and row[12]!='Master.' and row[3]>100:
        j11+=1
        if row[10]==0: i11+=1
        return(1)
DF['Surv']=DF.apply(funct, axis=1)
j=[j1,j2,j3,j4,j5,j6,j7,j8,j9,j10,j11]
i=[i1,i2,i3,i4,i5,i6,i7,i8,i9,i10,i11]
print(i)
for q in range(11):
    print(i[q]/j[q], end=' ')
print(pd.crosstab(DF.Surv,DF.Survived))
(891-153)/891